In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [2]:
# API Connection
from sodapy import Socrata

client = Socrata("data.cityofnewyork.us", "yUnYRTjXHWdK6ATxbXOsJR52v", timeout=60)

# First 20000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("qgea-i56i", limit=20000)
df = pd.DataFrame.from_records(results)

In [3]:
# define list of values from column ofns_desc and make a new list
ofns_desc = df['ofns_desc'].value_counts().index.tolist()
ofns_desc

['PETIT LARCENY',
 'HARRASSMENT 2',
 'ASSAULT 3 & RELATED OFFENSES',
 'GRAND LARCENY',
 'CRIMINAL MISCHIEF & RELATED OF',
 'FELONY ASSAULT',
 'VEHICLE AND TRAFFIC LAWS',
 'GRAND LARCENY OF MOTOR VEHICLE',
 'ROBBERY',
 'BURGLARY',
 'MISCELLANEOUS PENAL LAW',
 'OFF. AGNST PUB ORD SENSBLTY &',
 'DANGEROUS DRUGS',
 'SEX CRIMES',
 'OFFENSES AGAINST PUBLIC ADMINI',
 'DANGEROUS WEAPONS',
 'INTOXICATED & IMPAIRED DRIVING',
 'FORGERY',
 'THEFT-FRAUD',
 'OFFENSES INVOLVING FRAUD',
 'FRAUDS',
 'CRIMINAL TRESPASS',
 'POSSESSION OF STOLEN PROPERTY',
 'RAPE',
 'UNAUTHORIZED USE OF A VEHICLE',
 'ADMINISTRATIVE CODE',
 'OFFENSES AGAINST THE PERSON',
 'NYS LAWS-UNCLASSIFIED FELONY',
 'OTHER STATE LAWS (NON PENAL LA',
 'ARSON',
 'OTHER OFFENSES RELATED TO THEF',
 'MURDER & NON-NEGL. MANSLAUGHTER',
 "BURGLAR'S TOOLS",
 'OFFENSES AGAINST PUBLIC SAFETY',
 'FRAUDULENT ACCOSTING',
 'OTHER STATE LAWS',
 'GAMBLING',
 'THEFT OF SERVICES',
 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
 'CANNABIS RELATED OFFENSES',
 '

In [4]:
# First 500000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy. 
# Make a loop and put through api query for every value from ofns_desc list. 
# Include additional columns alongside cmplnt_fr_dt, cmplnt_fr_tm, and ofns_desc 

ofns_desc_list = df['ofns_desc'].value_counts().index.tolist()

# Set today's date
today = datetime.now().strftime("%Y-%m-%d")

# Create an empty list to store the results
results_list = []

# Loop through each value in the ofns_desc list
for ofns_desc_value in ofns_desc_list:
    # Escaping apostrophes in the value
    ofns_desc_value_escaped = ofns_desc_value.replace("'", "''")
    
    # Forming the query with escaped apostrophes
    query = f"ofns_desc='{ofns_desc_value_escaped}' AND cmplnt_fr_dt >= '2010-01-01' AND cmplnt_fr_dt <= '{today}'"
   
    # Query the API for the specific value from ofns_desc list and include additional columns
    results = client.get("qgea-i56i",
                         select="cmplnt_fr_dt, cmplnt_fr_tm, ofns_desc, boro_nm, susp_age_group, susp_race, susp_sex, vic_age_group, vic_race, vic_sex",
                         where=query,
                         limit=500000)  # Adjusted the limit to a realistic number
    
    # Convert the results to a DataFrame
    results_df = pd.DataFrame.from_records(results)
    
    # Append the results to the results_list
    results_list.append(results_df)

# Concatenate all the DataFrames in the results_list
final_df = pd.concat(results_list)


In [5]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4916471 entries, 0 to 125
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   cmplnt_fr_dt    object
 1   cmplnt_fr_tm    object
 2   ofns_desc       object
 3   boro_nm         object
 4   susp_age_group  object
 5   susp_race       object
 6   susp_sex        object
 7   vic_age_group   object
 8   vic_race        object
 9   vic_sex         object
dtypes: object(10)
memory usage: 412.6+ MB


In [6]:
# save the final_df to a csv file
final_df.to_csv('Resources/nypd_crime_data.csv', index=False)